In [1]:
import numpy as np
from scipy.interpolate import interp1d
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks, peak_prominences, argrelmin
import matplotlib.pyplot as plt



In [2]:
# Define standards for plots
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams['figure.dpi'] = 200
matplotlib.rcParams['lines.linewidth'] = 1.5

In [3]:
def GroenModel(U, t, deltaT, T, Ws, alpha, Kv):
    """ 
    function GroenModel

    INPUT:
        U = depth averaged flow velocity in m/s
        t = time in s
        deltaT = timestep in s
        T = Timescale in s
        Ws = 'fall velocity' in m/s 
        alpha = 'erosion parameter' in kg s/m^4
        Kv = vertical eddy diffusivity m^2/s

    OUTPUT:
        Conc = 'the depth-integrated sediment concentration'
        q = 'sediment transport in  'kg m/s'
    """
    
    Tend = t[-1]
    deltaTfix = Kv / Ws**2  # Maximum time step allowed
    
    if deltaTfix > deltaT:
        deltaTfix = deltaT
    else:
        deltaT = deltaTfix
        
    # Within the code deltaT can be time dependent. Most often deltaTfix will be used. 
    
    Nt = int(np.ceil(Tend/deltaT) + 1)
    deltaTnew = Tend / Nt

    tt = np.arange(0, Tend + deltaTnew, deltaTnew)
    
    set_interp = interp1d(t, U, kind = 'linear')
    Uf = set_interp(tt)
    
    E = np.zeros(Nt+1)
    D = np.zeros(Nt+1)
    C = np.zeros(Nt+1)
    
    # initialise solution at t=0
    tt[0] = t[0]
    C[0]  = 0.0
    
    for k in range(Nt):
        
        # ************************************************

        # Predictor step

        E[k] = alpha*Uf[k]**2
        D[k] = (Ws**2/Kv) * C[k]
        # New C will be caused by difference between erosion and deposition
        C[k+1] = C[k] + (E[k]-D[k])*deltaT
        
        # End of predictor step
        # *********************************************
        
        # *****************************************************************
        # Corrector step. See assignment. 
        
        E[k+1] = alpha*Uf[k+1]**2
        D[k+1] = (Ws**2/Kv)*C[k+1]
        C[k+1] = C[k] + 0.5*(E[k]+E[k+1]-D[k]-D[k+1])*deltaT
        
        # End of corrector step
        #*********************************************

        
    # Since we have calculated the solution on a new time vector, we have to interpolate the solution to the right time vector. 
    set_interp = interp1d(tt, C, kind = 'linear')
    Conc = set_interp(t)
    q = U*Conc
    
    return Conc, q


def HydroModel(t, Z, dZdt, H, dHdx, x, dx):
    """ 
    function HydroModel 

    INPUT:
        t = time
        Z = water level at seaward boundary
        dZdt = change of water level in time
        H = still water depth as a function of position x
        L = length of basin
        dx = griddistance

    OUTPUT:
        U = velocity (m/s)
    """
   
    # Initialisation
    Nx = len(x)
    Nt = len(t)
    L = x[-1]
     
    testPlane = np.cumsum(abs(dHdx)>0)
     
    # solve dZdt+d/dx Hu=0, assuming dZdx=0. So U * dHdx + (H+Z) dU/dx=-dZ/dt
    # U=0 at seaward boundary.
    
    U = np.zeros((Nx, Nt))
    if testPlane[-1] == 0:
        H0 = H[0]
        U[0:Nx, 0:Nt] = x * -dZdt/(H0+Z) 
    else:
        U[Nx-1, 0:Nt] = 0
        for px in range(Nx-1):
            U[px, 0:Nt] = (x[px] - L) * (-dZdt/(H[px]+Z))
     
    return U




def get_Q_arr(Ws=5e-4, alpha=1e-4, Kv=1e-2, phaseD1=0, phaseM2=0, phaseM4=np.pi/2):
    '''
    Function to get time, C and Q. 
    Input:
        Ws = 
        alpha = 
        Kv = 
    Output: 
        t = 
        C = 
        Q = 
    '''
    
    T = (12*60+25)*60        # We model only the M2 and M4 tide. Time is in seconds. 
    Tadapt = Kv/Ws**2
    Test = 3*Tadapt          # Estimated adaptation time scale to get to equilibrium C
    Tnt  = np.ceil(Test/T)
    Tend = 3*Tnt*T           # Depends on Ws and Kv. Enough time to Equilibrium C. 
    deltaT = 300             # Time step of 5 minutes
    t  = np.arange(0, Tend+deltaT, deltaT)
    Nt = len(t)
    
    ampD1 = 0            # in part 1 and 2 D1=0. Depending on your estuary, you might want to prescribe D1 for part 3. 
    ampM2 = 1
    ampM4 = 0.2
    
    Z = ampD1*np.sin(np.pi*t/T + phaseD1)+ ampM2*np.sin(2*np.pi*t/T + phaseM2)+ ampM4*np.sin(4*np.pi*t/T + phaseM4)          # Waterlevel prescribed as sine function. 
    dZdt = ampD1*1*np.pi/T*np.cos(np.pi*t/T+ phaseD1)+ ampM2*2*np.pi/T*np.cos(2*np.pi*t/T+ phaseM2)+ ampM4*4*np.pi/T*np.cos(4*np.pi*t/T + phaseM4)  # Flow velocity will behave as a cosine function. 
    
    L  = 1e4                     # We model a simple basin with a length of 10 km
    dx = 400                     # Grid distance
    x  = np.arange(0, L+dx, dx)  # x-coordinate. Seaward end is at x=L, landward end at x=0. 
    Nx = len(x)                  # array of 26 long
    H = 10-8e-4 * x             # Bottom profile. Linear sloping bottom. 2 m deep near landward boundary, 10 m deep near inlet. 
    dHdx = np.ones(Nx) * -8e-4

    U = HydroModel(t,Z,dZdt,H,dHdx,x,dx) # Calculating flow velocities 

    Cgroen = np.zeros((Nx, Nt))
    Qsgroen = np.zeros((Nx,Nt))
    
    for px in range(Nx):
        Cgroen[px,0:Nt],Qsgroen[px,0:Nt] = GroenModel(U[px,0:Nt],t,deltaT, T, Ws, alpha, Kv) #using groens model to find C and Q
        
        
    return t, Cgroen, U ,Qsgroen

 
# Nsteps = T/deltaT       # Nr of timesteps in one tidal cycle.


# calculate tidally averaged sediment transport (only averaging over last tidal cycle)
# final_cycle = int(-Nsteps)

# meanQsgroen = np.mean(get_Q_arr()[:,final_cycle:], 1)
def peaks_comparison(c, q):
    '''
    Function to find difference in peaks. 
        Input:
            c = the depth-integrated sediment concentration
            q = 'sediment transport in  'kg m/s'
        output:
            difference = difference between peaks
    '''
    peaks_C,bs = find_peaks(c, distance = 60) #distance is set such that the small peak before big peak isn't considered
    peaks_Q,bs = find_peaks(q, distance=60)
    
    sizeC = int(peaks_C.size)
    sizeQ = int(peaks_Q.size)
    
    
    sec_last_peak = peaks_C[-2]
    diff_arr = np.abs(sec_last_peak - peaks_Q)
    min_diff_arr  =np.argmin(diff_arr)
    difference = sec_last_peak - peaks_Q[min_diff_arr]
    
#     if sizeC == sizeQ:
#         differenceX = peaks_C[:-1] - peaks_Q[:-1]
#     elif sizeC > sizeQ:
#         differenceX = peaks_C[0:sizeQ] - peaks_Q[0:sizeQ]
#     else:
#         differenceX = peaks_C[0:sizeC] - peaks_Q[0:sizeC]
        
        
    return difference

def peaksmachine(scalar_arr): 
    '''
    Function to calculate difference in s between peaks for different values of scalar array, Ws, and Kv
    
    Input:
        scalar_arr: numpy array with scalars. Dont put any values lower than 0.2 in the array.
        Ws: standard values
        Kv: standard values
    Output:
        WC: dictionary, Values of C with variable W 
        WQ: dictionary. Values of Q with variable W
        KC: dictionary. Values of C with variable K
        KQ: dictionary. Values of Q with variable k.
        
    '''
    
    Ws = 5e-4
    Kv = 1e-2
    
    warr_dict = {}      #Empty dictionaries to store the different variables in
    karr_dict = {}

    for i,scalar in enumerate(scalar_arr): #This for loop calculates the C and Q values for different values of Ws and Kv
        WsScalar = Ws*scalar
        Wsvar  = get_Q_arr(Ws=WsScalar)
        warr_dict['wqc{0}'.format(i)] = Wsvar 

        KvScalar = Kv*scalar 
        Kvvar = get_Q_arr(Kv=KvScalar)
        karr_dict['kqc{0}'.format(i)] = Kvvar

        
  
    WC = {} #Empty final filtered dictionaries
    WU = {}
    KC = {}
    KU = {}
    

    for i,d in enumerate(warr_dict):
        
        WC['WC_{0}'.format(i)] = warr_dict[str(d)][1] #The first dictionaries are the C values
        WC['WT_{0}'.format(i)] = warr_dict[str(d)][0] #Time array
        WU['WQ_{0}'.format(i)] = warr_dict[str(d)][2] #The second values are the U values

    for i,k in enumerate(karr_dict):
        
        KC['KC_{0}'.format(i)] = karr_dict[str(k)][1] #Same is done for different K
        KC['KT_{0}'.format(i)] = karr_dict[str(k)][0]
        KU['KQ_{0}'.format(i)] = karr_dict[str(k)][2]
        
        
    return WC, WU, KC, KU
    